# Lecture 3 演習(アンケート結果の可視化)

第１回講義にて、簡単なアンケートを取りました。教員は、この結果はエクセルとしてダウンロードできます。

エクセル形式で保存されたアンケート結果を集計分析する例を、できるだけインタラクティブに再現してみました。

## エクセルを読む
最初にエクセル形式のファイル(ここでは*.xlsという古い形式のデータを読みます)

ファイルはmanabaに掲載しますので、ダウンロードしてからCorab環境にuploadしてください。

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_excel('data/query-66083-66917.xls',skiprows=4)
df

Colab環境では失敗しましたね。どうもxlrdというパッケージのバージョンが1.2.0以上である必要があるようです。

In [ ]:
!pip install xlrd==1.2.0

カーネルをリスタートして、再度読み込みます。

In [ ]:
df = pd.read_excel('data/query-66083-66917.xls',skiprows=4)
df

どうも第１行が、カラム名で、第２行はその説明、３行目以降が一人一人のアンケートのようです。

とりあえず２つを行います
- 取り扱うコラムは#回答1.1～回答1.4ですので、名前をつけなおします。(df.columnsにリストを代入すればコラム名は付け替えられました)
- 最初の２行を読み飛ばします(行に対するスライスを使ってます。)

In [ ]:
df.columns=['col0','col1','col2','col3','col4','col5','col6','q1.1','q1.2','q1.3','q1.4']
df=df[2:]
df.head()

col5が'未提出'な行**以外**を絞り込み、さらにq1.1～q1.4のみを残します

In [ ]:
df2=df[df.iloc[:,5]!='未提出']
df2=df2[['q1.1','q1.2','q1.3','q1.4']]
df2

どうも最後の行がNaNで埋まっています、これは削除しましょう。

In [ ]:
df2=df2.dropna(how='all')
df2

## データの概要
さて質問は
- q1.1: 統計を学んだ経験について（複数選択）
- q1.2: 使ったことのあるプログラミング言語（複数選択)
- q1.3: Pythonを使った経験（３択）
- q1.4: データサイエンスに期待する項目は？(複数選択)

このように、複数選択可能なものがあり'1,2,3'のような文字列で複数選択結果が記録されています。

## q1.2　使ったことのあるプログラミング言語
まずは、複数選択された結果を含めて、使ったことのあるプログラミング言語の延べ数を集計します。

最初に、'q1.2'の列を取り出します。

In [ ]:
df2['q1.2'].values

'1,2,3'のような文字列を特定のセパレータ(ここでは',')でばらしたリストにするのにsplit()関数を追加います。

In [ ]:
hoge='1,2,3'
hogehoge=hoge.split(',')
hogehoge

そて、q1.2列の値のリストに対して、それぞれの要素をsplit関数でバラバラにして、それを１つのリストとして集約します。

ここでは**残念ながら** for文を使っています。

In [ ]:
ret=[]
q2list= df2['q1.2'].values
for i in range(len(q2list)):
  ret.extend(q2list[i].split(','))
ret

さあ、ここで、得られたリストの値は、カテゴリー変数とよばれれる変数です、'1'～'10'のカテゴリー毎に出現件数を集計するのにcollection.Counterを使います。

In [ ]:
import collections

c=collections.Counter(ret)
c

Counterは一種の辞書dictになっています。ここでキーを'1','2'ではなくて、設問にあった選択枝'VBA','C/C++'で付け替えます。

popして代入することで１行で付け替えができることに注意ください。

In [ ]:
c['VBA']=c.pop('1')
c['C/C++']=c.pop('2')
c['Python']=c.pop('3')
c['Ruby']=c.pop('4')
c['Java']=c.pop('5')
c['FORTRAN']=c.pop('6')
c['COBOL']=c.pop('7')
c['Basic']=c.pop('8')
c['Other']=c.pop('9')
c['Non']=c.pop('10')
c

In [ ]:
c.items()

In [ ]:
c.values()

## 円グラフの表示
それでは円グラフを描いてみましょう

In [ ]:
import matplotlib.pyplot as plt

plt.pie(c.values(),labels=c.keys(), counterclock=False, startangle=90, autopct="%.1f%%",
        pctdistance=0.7)
plt.title('Programming Language  Experience')

In [ ]:
len(q2list)

## q1.3 Pythonを使った経験
これは、３択なので簡単です。

In [ ]:
df2['q1.3']

In [ ]:
c3=collections.Counter(df2['q1.3'])
c3

In [ ]:
c3['over 3years']=c3.pop('1')
c3['less than 3years']=c3.pop('2')
c3['no python']=c3.pop('3')
c3

In [ ]:
plt.rcParams['font.family'] = 'IPAexGothic'
plt.pie(c3.values(),labels=c3.keys(), counterclock=False, startangle=90, autopct="%.1f%%",
        pctdistance=0.7)
plt.title('Python Experience')

## q1.1　統計を学んだ経験
これも、複数選択された結果を集計します。


In [ ]:
df2['q1.1']

In [ ]:
q1list= df2['q1.1'].ravel()
ret=[]
for i in range(len(q1list)):
  ret.extend(q1list[i].split(','))
ret

In [ ]:
c1=collections.Counter(ret)
c1

In [ ]:
c1['HighSchool']=c1.pop('1')
c1['University']=c1.pop('2')
c1['GradSchool']=c1.pop('3')
c1['OJT']=c1.pop('4')
c1['SelfTaught']=c1.pop('5')
c1['NoStatistics']=c1.pop('6')
c1['Other']=c1.pop('7')

In [ ]:
c1

In [ ]:
plt.pie(c1.values(),labels=c1.keys(), counterclock=False, startangle=90, autopct="%.1f%%",
        pctdistance=0.7)
plt.title('Statistic Course Experience')